In [2]:
import pandas as pd, numpy as np, matplotlib.pylab as plt
import MySQLdb

In [3]:
# Open database connection
conn = MySQLdb.connect(host='localhost',user='root',passwd='',db='qyw', charset='utf8')

In [4]:
user_visits = pd.read_sql('''
    SELECT USER_ID, OPERATE_TIME, PAGE_NAME, ELEMENT_NAME FROM qyw.qyw_5th_visit ORDER BY USER_ID, OPERATE_TIME;
    ''', con=conn)
user_visits.insert(0, 'CASE_ID','')

In [31]:
import datetime
unique_users = user_visits['USER_ID'].unique()
for user_id in unique_users:
    single_user_visits = user_visits[user_visits['USER_ID'] == user_id].sort_values(['OPERATE_TIME'],0,[True])
    size = len(single_user_visits.index)
    start = single_user_visits.index[0]
    date = None
    cid = 0
    for index, row in single_user_visits.iterrows():
        start_operate_time = row['OPERATE_TIME']
        cur_date = start_operate_time.strftime('%Y-%m-%d')
        if not date:
            date = cur_date
        end_operate_time = start_operate_time
        next_date = cur_date
        if index < start+size-1:
            end_operate_time = single_user_visits.loc[index+1, 'OPERATE_TIME']
            next_date = end_operate_time.strftime('%Y-%m-%d')
        interval = end_operate_time - start_operate_time
        interval_secs = interval.total_seconds()
        case_id = str(row['USER_ID'])+'@'+date+'_'+str(cid)
        single_user_visits.ix[index, 'CASE_ID'] = case_id
        if interval_secs > 5*60:
            if date != next_date:
                cid = 0
                date = next_date
            else:
                cid += 1
    single_user_visits.to_sql('qyw_5th_event', conn, flavor='mysql', if_exists='append', index=False)

In [ ]:
# single_user_visits = user_visits.ix[0:3,:]
# single_user_visits.iloc[0,0] = 1
# single_user_visits == user_visits.ix[0:3,:]
# user_visits
# single_user_idx = (user_visits['USER_ID'] == 27846) & (user_visits['PAGE_NAME'] == u'首页')
# single_user_visits = user_visits.ix[single_user_idx]
# single_user_visits.ix[0,0] = 'abc'
# single_user_visits == user_visits.ix[single_user_idx]

In [ ]:
foo = pd.DataFrame(np.random.random((13000,5)))
bar = foo.iloc[0:3,0:5]
bar.ix[0,0] = 0
foo
# bar == foo.iloc[3:5,1:4]